In [2]:
import pickle
import glob, os
import midi
from random import randint
import difflib
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [5]:
class NoteEvent:
    tickDuration =  0
    channel = 0
    data = 0
    def __init__(self, tickDuration, channel, data):
        self.tickDuration = tickDuration
        self.channel = channel
        self.data = data
    def getInfo(self):
        return ("tickDuration: %d, channel = %d, data = %d"%(self.tickDuration,self.channel,self.data))
        
def getNoteEvents(pattern):
    counter = 0
    listOfNoteEvents = []
    for track in pattern:
        for m in track:
            typeM = str(type(m))
            if(typeM.find('NoteOnEvent')!=-1):
                if(m.data[1]>0):
                    currentNote = NoteEvent(m.tick,m.channel,m.data[0])
                    listOfNoteEvents.append(currentNote)
                else:
                    #Assumption: MIDI file is a monophonic and one channel recording
                    currentNote = listOfNoteEvents[counter]
                    currentNote.tickDuration = m.tick - currentNote.tickDuration
                    counter+=1
    return listOfNoteEvents

def getNoteData(listOfNoteEvents):
    listOfNotesData = []
    for e in listOfNoteEvents:
        listOfNotesData.append(e.data)
    return listOfNotesData

def getUDS(listOfNoteData):
    pointer = 1
    listOfUDS = []
    uds='s'
    while(pointer<len(listOfNoteData)-1):
        if(listOfNoteData[pointer]>listOfNoteData[pointer-1]):
            uds='u'
        elif(listOfNoteData[pointer]<listOfNoteData[pointer-1]):
            uds='d'
        pointer+=1
        listOfUDS.append(uds)
    return listOfUDS

def pattern2UDS(pattern):
    listOfNoteEvents = getNoteEvents(pattern)
    listOfNoteData = getNoteData(listOfNoteEvents)
    listOfUDS = getUDS(listOfNoteData)
    udsString = ''.join(listOfUDS)
#    print (','.join(str(e) for e in listOfNoteData))
#    print (','.join(listOfUDS))
    return udsString

In [6]:
if __name__ == "__main__":
    dataDir = '/Users/maureen/Documents/Study/RWTH_Aachen/LabMultimedia/QueryByHumming/MIDI/MIR-QBSH-corpus/'
    os.chdir(dataDir)
    midiFiles = glob.glob("midiFile/*.mid")
    udsFile = dataDir + 'listOfUDS.pickle'
    listOfUDSString = []
    for file in midiFiles:
        dataFullpath = dataDir + file
        pattern = midi.read_midifile(dataFullpath)
        udsString= pattern2UDS(pattern)
        listOfUDSString.append(udsString)
    pickle.dump(listOfUDSString, open( udsFile, "wb" ))

In [33]:
minLen = 5
#pointer = randint(0,len(listOfUDSString)-1)
hum = listOfUDSString[pointer]
#lenStr = randint(minLen,len(hum))
#startIdx = randint(0,len(hum)-lenStr)

pointer = 45
lenStr = 134
startIdx = 86
hum = hum[startIdx:startIdx+lenStr]
print("Pointer: ",pointer)
print("LenStr: ",lenStr)
print("StartIdx: ",startIdx)
print("OriginalString: ",listOfUDSString[pointer])
print("Hum: ",hum)
bestPartialRatio = -1
bestRatio = -1
idxMatch = -1
counter = 0
for uds in listOfUDSString:
    print(counter)
    partialRatio = fuzz.partial_ratio(hum,uds)
    ratio = fuzz.ratio(hum,uds)
    if(bestPartialRatio < partialRatio):
        bestPartialRatio = partialRatio
        bestRatio = ratio
        idxMatch = counter
    elif((bestPartialRatio == partialRatio) and (bestRatio < ratio)):
        bestRatio = ratio
        idxMatch = counter
    counter+=1    
    print("FullRatio:", ratio)
    print("PartialRatio: ", partialRatio)
    print("==============")
if(idxMatch>=0):
    print("IdxMatch: ", idxMatch)
    print("FullUDS: ", listOfUDSString[idxMatch])
process.extract(hum, listOfUDSString, limit=2)
#best = difflib.get_close_matches(listOfUDSString[pointer], listOfUDSString)


#print (best)

Pointer:  45
LenStr:  134
StartIdx:  86
OriginalString:  uuuudddddduuuuddduuuduuuuddddduuuudddduuuudduuudddduddduduuuudduuuddddudddduuuuuudddddduuuuddduuuduuuuddddduuuudddduuuudduuudddduddduduuuudduuuddddudddduuuuuudddddduuuuddduuuduuuuddddduuuudddduuuudduuudddduddduduuuudduuuddddudddduuuuudduuuddddudddduuuuudduuuddddudddduuuuudduuuddddudddd
Hum:  duuuuddduuuduuuuddddduuuudddduuuudduuudddduddduduuuudduuuddddudddduuuuuudddddduuuuddduuuduuuuddddduuuudddduuuudduuudddduddduduuuudduuu
0
FullRatio: 31
PartialRatio:  85
1
FullRatio: 49
PartialRatio:  72
2
FullRatio: 37
PartialRatio:  75
3
FullRatio: 39
PartialRatio:  79
4
FullRatio: 52
PartialRatio:  69
5
FullRatio: 66
PartialRatio:  70
6
FullRatio: 36
PartialRatio:  84
7
FullRatio: 34
PartialRatio:  66
8
FullRatio: 30
PartialRatio:  84
9
FullRatio: 42
PartialRatio:  72
10
FullRatio: 33
PartialRatio:  73
11
FullRatio: 35
PartialRatio:  80
12
FullRatio: 51
PartialRatio:  64
13
FullRatio: 32
PartialRatio:  75
14
FullRatio: 30
PartialRatio:  69


[('uuudduudduuuudduudddduuuuuuddddduuuddduuuduuuudduudduuuudduu', 76),
 ('ssdduuuuddddduuuuddddduuuuduuddd', 76)]

In [21]:
#print ('\n\n'.join(listOfUDSString))